## Setup: Imports and Environment Configuration

In [12]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import json
from tqdm import tqdm

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

# Load environment variables from .env file
#load_dotenv(project_root / '.env')

# Set OpenAI API key from environment
#os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', os.environ.get('OPENAI_API_KEY', ''))
#if not os.environ.get('OPENAI_API_KEY'):
    #raise ValueError("OPENAI_API_KEY not found in .env file or environment variables")

# Import project modules
from src.utils.config_loader import load_config
from src.ingestion.pdf_loader import load_pdf, clean_text
from src.ingestion.chunker import chunk_text
from src.ingestion.qa_generator import generate_qa_pairs
from src.ingestion.dataset_writer import save_to_jsonl, split_dataset, filter_valid_pairs

print("✓ Imports and environment setup complete")
#print(f"✓ Project root: {project_root}")
#print(f"✓ OpenAI API key loaded: {'Yes' if os.environ.get('OPENAI_API_KEY') else 'No'}")

✓ Imports and environment setup complete


In [13]:
from google.colab import userdata
import os

# Retrieve API key from Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')

# Set environment variable for OpenAI client
os.environ['OPENAI_API_KEY'] = openai_api_key

print("✓ OpenAI API key loaded and set as environment variable")

✓ OpenAI API key loaded and set as environment variable


## Step 1: Load Configuration

In [14]:
# Load configuration
config_path = project_root / 'config' / 'config.yaml'
config = load_config(config_path)

# Extract data factory configuration
df_config = config['data_factory']
ingestion_config = df_config['ingestion']
chunking_config = df_config['chunking']
generation_config = df_config['generation']
dataset_config = df_config['dataset']

# Get paths
raw_data_path = project_root / config['environment']['paths']['raw_data']
output_path = project_root / config['environment']['paths']['data_dir'] / 'output'
output_path.mkdir(parents=True, exist_ok=True)

# Verify raw_data_path exists
if not raw_data_path.exists():
    raise FileNotFoundError(f"Raw data directory not found: {raw_data_path}")

# Get PDF filename from config, with fallback to actual file
pdf_filename = config['project']['document']
pdf_path = raw_data_path / pdf_filename

# If configured PDF doesn't exist, try to find any PDF in the directory
if not pdf_path.exists():
    pdf_files = list(raw_data_path.glob('*.pdf'))
    if pdf_files:
        pdf_path = pdf_files[0]
        pdf_filename = pdf_path.name
        print(f"⚠ Config PDF '{config['project']['document']}' not found, using: {pdf_filename}")
    else:
        raise FileNotFoundError(f"PDF file not found in {raw_data_path}. Expected: {pdf_path}")

# Verify PDF exists
if not pdf_path.exists():
    raise FileNotFoundError(f"PDF file not found: {pdf_path}")

print("✓ Configuration loaded")
print(f"  Raw data directory: {raw_data_path} (exists: {raw_data_path.exists()})")
print(f"  PDF path: {pdf_path} (exists: {pdf_path.exists()})")
print(f"  Output path: {output_path}")
print(f"  Chunk size: {chunking_config['chunk_size']} characters")
print(f"  Questions per chunk: {generation_config['questions_per_chunk']}")
print(f"  Categories: {', '.join(generation_config['categories'])}")

✓ Configuration loaded
  Raw data directory: /content/operation-ledger-mind/data/raw (exists: True)
  PDF path: /content/operation-ledger-mind/data/raw/2024-Annual-Report.pdf (exists: True)
  Output path: /content/operation-ledger-mind/data/output
  Chunk size: 1500 characters
  Questions per chunk: 10
  Categories: hard_facts, strategic_summary, stylistic_creative


## Step 2: PDF Ingestion & Cleaning

In [ ]:
# Load PDF
print(f"Loading PDF from: {pdf_path}")

try:
    raw_text = load_pdf(pdf_path)
    print(f"✓ PDF loaded: {len(raw_text):,} characters")
except ValueError as e:
    print(f"Error loading PDF: {e}. Attempting to install pdfplumber...")
    %pip install pdfplumber  # Install pdfplumber if it's missing
    raw_text = load_pdf(pdf_path) # Retry loading after installation
    print(f"✓ PDF loaded after installing pdfplumber: {len(raw_text):,} characters")

# Clean text
print("\nCleaning text...")
cleaned_text = clean_text(
    raw_text,
    remove_headers=ingestion_config['remove_headers'],
    remove_footers=ingestion_config['remove_footers'],
    normalize_whitespace=ingestion_config['normalize_whitespace']
)
print(f"✓ Text cleaned: {len(cleaned_text):,} characters")
print(f"  Reduction: {len(raw_text) - len(cleaned_text):,} characters ({100*(len(raw_text)-len(cleaned_text))/len(raw_text):.1f}%")

Loading PDF from: /content/operation-ledger-mind/data/raw/2024-Annual-Report.pdf
Error loading PDF: Failed to extract text from PDF. Please install one of: PyMuPDF (fitz), pdfplumber, or PyPDF2. Attempting to install pdfplumber...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 142.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 114.1 MB/s eta 0:00:00
✓ PDF loaded after installing pdfplumber: 620,266 characters

Cleaning text...
✓ Text cleaned: 619,806 characters
  Reduction: 460 characters (0.1%


## Step 3: Chunking Strategy

In [16]:
# Chunk the document
print("Chunking document...")
chunks = chunk_text(
    cleaned_text,
    chunk_size=chunking_config['chunk_size'],
    overlap=chunking_config['overlap']
)

print(f"✓ Document chunked into {len(chunks)} chunks")
print(f"  Average chunk size: {sum(len(c['text']) for c in chunks) / len(chunks):.0f} characters")
print(f"  Total characters: {sum(len(c['text']) for c in chunks):,}")

# Display first chunk preview
if chunks:
    print(f"\nFirst chunk preview (chunk_id={chunks[0]['chunk_id']}):")
    print(f"  {chunks[0]['text'][:200]}...")

Chunking document...
✓ Document chunked into 414 chunks
  Average chunk size: 1497 characters
  Total characters: 619,806

First chunk preview (chunk_id=0):
  On Our Way
Uber’s Mission
We reimagine the way the world moves for the better
We are Uber. The go-getters. The kind of people who are relentless about our
mission to help people go anywhere and get an...


## Step 4: Q/A Generation Loop

In [17]:
# Prepare LLM configurations
question_llm_config = {
    'provider': generation_config['question_llm']['provider'],
    'model': generation_config['question_llm']['model'],
    'temperature': generation_config['question_llm']['temperature'],
    'max_tokens': generation_config['question_llm']['max_tokens'],
    'timeout_seconds': config['providers']['openai']['timeout_seconds'],
    'max_retries': config['providers']['openai']['max_retries']
}

answer_llm_config = {
    'provider': generation_config['answer_llm']['provider'],
    'model': generation_config['answer_llm']['model'],
    'temperature': generation_config['answer_llm']['temperature'],
    'max_tokens': generation_config['answer_llm']['max_tokens'],
    'timeout_seconds': config['providers']['openai']['timeout_seconds'],
    'max_retries': config['providers']['openai']['max_retries']
}

categories = generation_config['categories']

print("LLM Configuration:")
print(f"  Question LLM: {question_llm_config['model']} (temp={question_llm_config['temperature']})")
print(f"  Answer LLM: {answer_llm_config['model']} (temp={answer_llm_config['temperature']})")
print(f"  Categories: {', '.join(categories)}")

LLM Configuration:
  Question LLM: gpt-4o-mini (temp=0.3)
  Answer LLM: gpt-4o (temp=0.2)
  Categories: hard_facts, strategic_summary, stylistic_creative


In [18]:
# Generate Q/A pairs for each chunk
all_qa_pairs = []
failed_chunks = []

print(f"\nGenerating Q/A pairs for {len(chunks)} chunks...")
print("=" * 60)

for chunk in tqdm(chunks, desc="Processing chunks"):
    chunk_id = chunk['chunk_id']

    try:
        # Generate Q/A pairs for this chunk
        qa_pairs = generate_qa_pairs(
            chunk=chunk,
            question_llm_config=question_llm_config,
            answer_llm_config=answer_llm_config,
            categories=categories
        )

        # Add chunk text reference to each pair (optional, for debugging)
        for pair in qa_pairs:
            pair['chunk_text'] = chunk['text'][:500]  # Store first 500 chars for reference

        all_qa_pairs.extend(qa_pairs)

        # Progress update
        if (chunk_id + 1) % 5 == 0:
            print(f"  Processed {chunk_id + 1}/{len(chunks)} chunks - {len(all_qa_pairs)} Q/A pairs generated")

    except Exception as e:
        print(f"\n⚠ Error processing chunk {chunk_id}: {str(e)}")
        failed_chunks.append(chunk_id)
        continue

print("=" * 60)
print(f"\n✓ Q/A generation complete!")
print(f"  Total Q/A pairs: {len(all_qa_pairs)}")
print(f"  Expected: {len(chunks) * generation_config['questions_per_chunk']}")
print(f"  Failed chunks: {len(failed_chunks)}")
if failed_chunks:
    print(f"  Failed chunk IDs: {failed_chunks}")


Generating Q/A pairs for 414 chunks...


Processing chunks:   1%|          | 5/414 [00:44<54:59,  8.07s/it]  

  Processed 5/414 chunks - 50 Q/A pairs generated


Processing chunks:   2%|▏         | 10/414 [01:24<53:06,  7.89s/it]

  Processed 10/414 chunks - 100 Q/A pairs generated


...

  Processed 405/414 chunks - 4050 Q/A pairs generated


Processing chunks:  99%|█████████▉| 410/414 [58:56<00:23,  5.99s/it]

  Processed 410/414 chunks - 4100 Q/A pairs generated


Processing chunks: 100%|██████████| 414/414 [59:21<00:00,  8.60s/it]


✓ Q/A generation complete!
  Total Q/A pairs: 4140
  Expected: 4140
  Failed chunks: 0


In [19]:
# Filter out invalid pairs and show statistics
valid_pairs = filter_valid_pairs(all_qa_pairs)
invalid_count = len(all_qa_pairs) - len(valid_pairs)

print(f"Quality Check:")
print(f"  Valid pairs: {len(valid_pairs)}")
print(f"  Invalid pairs removed: {invalid_count}")

# Category distribution
category_counts = {}
for pair in valid_pairs:
    category = pair.get('category', 'unknown')
    category_counts[category] = category_counts.get(category, 0) + 1

print(f"\nCategory Distribution:")
for category, count in category_counts.items():
    percentage = 100 * count / len(valid_pairs) if valid_pairs else 0
    print(f"  {category}: {count} ({percentage:.1f}%)")

# Display sample Q/A pairs
if valid_pairs:
    print(f"\nSample Q/A Pairs:")
    for i, pair in enumerate(valid_pairs[:3], 1):
        print(f"\n  Example {i} ({pair.get('category', 'unknown')}):")
        print(f"    Q: {pair['question'][:100]}...")
        print(f"    A: {pair['answer'][:100]}...")

Quality Check:
  Valid pairs: 4140
  Invalid pairs removed: 0

Category Distribution:
  strategic_summary: 708 (17.1%)
  hard_facts: 2533 (61.2%)
  stylistic_creative: 899 (21.7%)

Sample Q/A Pairs:

  Example 1 (strategic_summary):
    Q: What is Uber's mission as stated in the annual report?...
    A: We reimagine the way the world moves for the better...

  Example 2 (hard_facts):
    Q: For which fiscal year does this annual report apply?...
    A: For the fiscal year ended December 31, 2024...

  Example 3 (hard_facts):
    Q: What is the commission file number associated with Uber Technologies, Inc.?...
    A: 001-38902...


## Step 5: Dataset Splitting & Storage

In [21]:
# Split dataset into train and test sets
print("Splitting dataset...")
train_pairs, test_pairs = split_dataset(
    all_pairs=valid_pairs,
    train_split=dataset_config['train_split'],
    shuffle=dataset_config['shuffle_before_split'],
    seed=dataset_config['seed']
)

print(f"✓ Dataset split:")
print(f"  Train set: {len(train_pairs)} pairs ({100*len(train_pairs)/len(valid_pairs):.1f}%)")
print(f"  Test set: {len(test_pairs)} pairs ({100*len(test_pairs)/len(valid_pairs):.1f}%)")

Splitting dataset...
✓ Dataset split:
  Train set: 3312 pairs (80.0%)
  Test set: 828 pairs (20.0%)


In [22]:
# Save datasets to JSONL files
train_file = output_path / dataset_config['train_file']
test_file = output_path / dataset_config['test_file']

print(f"\nSaving datasets...")
save_to_jsonl(train_pairs, train_file)
save_to_jsonl(test_pairs, test_file)

print(f"✓ Datasets saved:")
print(f"  Train: {train_file}")
print(f"  Test: {test_file}")

# Verify files
print(f"\nFile verification:")
print(f"  Train file size: {train_file.stat().st_size / 1024:.1f} KB")
print(f"  Test file size: {test_file.stat().st_size / 1024:.1f} KB")


Saving datasets...
✓ Datasets saved:
  Train: /content/operation-ledger-mind/data/output/train.jsonl
  Test: /content/operation-ledger-mind/data/output/golden_test_set.jsonl

File verification:
  Train file size: 2620.1 KB
  Test file size: 659.1 KB


## Step 6: Verification & Summary

In [23]:
# Load and verify a few samples from each file
print("Verifying saved files...\n")

# Check train file
with open(train_file, 'r', encoding='utf-8') as f:
    train_samples = [json.loads(line) for line in f.readlines()[:3]]

print(f"Train file samples (first 3):")
for i, sample in enumerate(train_samples, 1):
    print(f"  {i}. Q: {sample['question'][:80]}...")
    print(f"     Category: {sample.get('category', 'N/A')}")

# Check test file
with open(test_file, 'r', encoding='utf-8') as f:
    test_samples = [json.loads(line) for line in f.readlines()[:3]]

print(f"\nTest file samples (first 3):")
for i, sample in enumerate(test_samples, 1):
    print(f"  {i}. Q: {sample['question'][:80]}...")
    print(f"     Category: {sample.get('category', 'N/A')}")

print("\n" + "=" * 60)
print("✓ DATA FACTORY PIPELINE COMPLETE!")
print("=" * 60)
print(f"\nSummary:")
print(f"  • PDF processed: {pdf_filename}")
print(f"  • Chunks created: {len(chunks)}")
print(f"  • Total Q/A pairs: {len(valid_pairs)}")
print(f"  • Train set: {len(train_pairs)} pairs")
print(f"  • Test set: {len(test_pairs)} pairs")
print(f"  • Output directory: {output_path}")

Verifying saved files...

Train file samples (first 3):
  1. Q: What valuation model is used to determine the grant-date fair value of market-ba...
     Category: strategic_summary
  2. Q: What is the tone of the report regarding the urgency of addressing climate chang...
     Category: stylistic_creative
  3. Q: What does Adjusted EBITDA represent in the context of the company's financial me...
     Category: hard_facts

Test file samples (first 3):
  1. Q: In what ways could the potential disclosure of personal data impact the company'...
     Category: hard_facts
  2. Q: What types of assets are considered when allocating the fair value of purchase c...
     Category: hard_facts
  3. Q: How does the company plan to evaluate the impact of the new accounting standards...
     Category: strategic_summary

✓ DATA FACTORY PIPELINE COMPLETE!

Summary:
  • PDF processed: 2024-Annual-Report.pdf
  • Chunks created: 414
  • Total Q/A pairs: 4140
  • Train set: 3312 pairs
  • Test set: 828 pai